In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import os, time, pickle
from tqdm.notebook import tqdm

from sl_model import SL512, DSODSL512
from ssd_data import InputGenerator
from sl_utils import PriorUtil
from sl_training import SegLinkLoss, SegLinkFocalLoss

from utils.model import load_weights, calc_memory_usage
from utils.training import MetricUtility

### Data

In [ ]:
from data_synthtext import GTUtility

file_name = 'gt_util_synthtext_seglink.pkl'
with open(file_name, 'rb') as f:
    gt_util = pickle.load(f)
gt_util_train, gt_util_val = gt_util.split(0.9)

print(gt_util_train)

### Model

In [ ]:
# SegLink
model = SL512()
weights_path = './models/ssd512_voc_weights_fixed.hdf5'
#weights_path = '~/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
batch_size = 24
experiment = 'sl512_synthtext'
#experiment = 'sl512fl_synthtext'

In [ ]:
# SegLink + DenseNet
model = DSODSL512()
#model = DSODSL512(activation='leaky_relu')
weights_path = None
batch_size = 6
experiment = 'dsodsl512_synthtext'

In [ ]:
# SegLink + ResNet
from sl_model import SL512_resnet
model = SL512_resnet()
weights_path = None
batch_size = 10
experiment = 'sl512_resnet_synthtext'
# NOTE: did not converge and I had no time for further investigations

In [ ]:
freeze = []

if weights_path is not None:
    if weights_path.find('ssd512') > -1:
        layer_list = [
            'conv1_1', 'conv1_2',
            'conv2_1', 'conv2_2',
            'conv3_1', 'conv3_2', 'conv3_3',
            'conv4_1', 'conv4_2', 'conv4_3',
            'conv5_1', 'conv5_2', 'conv5_3',
            'fc6', 'fc7',
            'conv6_1', 'conv6_2',
            'conv7_1', 'conv7_2',
            'conv8_1', 'conv8_2',
            'conv9_1', 'conv9_2',
        ]
        freeze = [
            'conv1_1', 'conv1_2',
            'conv2_1', 'conv2_2',
            'conv3_1', 'conv3_2', 'conv3_3',
            #'conv4_1', 'conv4_2', 'conv4_3',
            #'conv5_1', 'conv5_2', 'conv5_3',
        ]
        
        load_weights(model, weights_path, layer_list)
        for layer in model.layers:
            layer.trainable = not layer.name in freeze
    else:
        load_weights(model, weights_path)

prior_util = PriorUtil(model)

### Training

In [ ]:
epochs = 100
initial_epoch = 0

#optimizer = keras.optimizers.SGD(lr=1e-3, momentum=0.9, decay=0, nesterov=True)
optimizer = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0)

#loss = SegLinkLoss(lambda_offsets=1.0, lambda_links=1.0, neg_pos_ratio=3.0)
loss = SegLinkFocalLoss(lambda_segments=100.0, lambda_offsets=1.0, lambda_links=100.0, gamma_segments=2, gamma_links=2)

#regularizer = None
regularizer = keras.regularizers.l2(5e-4) # None if disabled

gen_train = InputGenerator(gt_util_train, prior_util, batch_size, model.image_size, augmentation=False)
gen_val = InputGenerator(gt_util_val, prior_util, batch_size, model.image_size, augmentation=False)


dataset_train, dataset_val = gen_train.get_dataset(), gen_val.get_dataset()
iterator_train, iterator_val = iter(dataset_train), iter(dataset_val)

checkdir = './checkpoints/' + time.strftime('%Y%m%d%H%M') + '_' + experiment

if not os.path.exists(checkdir):
    os.makedirs(checkdir)

with open(checkdir+'/source.py','wb') as f:
    source = ''.join(['# In[%i]\n%s\n\n' % (i, In[i]) for i in range(len(In))])
    f.write(source.encode())

print(checkdir)

for l in model.layers:
    l.trainable = not l.name in freeze
    if regularizer and l.__class__.__name__.startswith('Conv'):
        model.add_loss(lambda l=l: regularizer(l.kernel))

metric_util = MetricUtility(loss.metric_names, logdir=checkdir)

@tf.function
def step(x, y_true, training=False):
    if training:
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            metric_values = loss.compute(y_true, y_pred)
            total_loss = metric_values['loss']
            if len(model.losses):
                total_loss += tf.add_n(model.losses)
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    else:
        y_pred = model(x, training=True)
        metric_values = loss.compute(y_true, y_pred)
    return metric_values

#tf.profiler.experimental.start('./tblog')

for k in tqdm(range(initial_epoch, epochs), 'total', leave=False):
    print('\nepoch %i/%i' % (k+1, epochs))
    metric_util.on_epoch_begin()

    for i in tqdm(range(gen_train.num_batches//4), 'training', leave=False):
        x, y_true = next(iterator_train)
        metric_values = step(x, y_true, training=True)
        metric_util.update(metric_values, training=True)
        #if i == 100: break
    
    model.save_weights(checkdir+'/weights.%03i.h5' % (k+1,))

    for i in tqdm(range(gen_val.num_batches), 'validation', leave=False):
        x, y_true = next(iterator_val)
        metric_values = step(x, y_true, training=False)
        metric_util.update(metric_values, training=False)
        #if i == 10: break

    metric_util.on_epoch_end(verbose=1)
    #if k == 1: break

#tf.profiler.experimental.stop()